In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
mnist

In [3]:
mnist.train.labels

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [6]:
mnist.train.images[1].shape

(784,)

In [8]:
import tensorflow as tf

In [34]:
x=tf.placeholder(tf.float32, [None, 784])
#Here , None means that a dimension can be of any length

In [35]:
W =tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [36]:
y = tf.nn.softmax(tf.matmul(x,W)+b)

In [37]:
y_ = tf.placeholder(tf.float32, [None,10])

In [38]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))

In [39]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [40]:
init = tf.initialize_all_variables()

In [41]:
sess = tf.Session()

In [42]:
sess.run(init)

In [50]:
for i in range(1000):
    batch_xs , batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys})

In [46]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))

In [47]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [51]:
sess.run(accuracy , feed_dict={x:mnist.test.images, y_:mnist.test.labels})

0.91960001

In [53]:
def weight_variable(shape):
    initial =tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_varaible(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [54]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [55]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [64]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_varaible([32])

In [65]:
x_image = tf.reshape(x,[-1,28,28,1])

In [66]:
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [67]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_varaible([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [70]:
w_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_varaible([1024])

h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)

In [71]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [72]:
w_fc2 = weight_variable([1024,10])
b_fc2 = weight_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)

In [74]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(tf.log(y_conv)*y_,reduction_indices=[1]))

In [76]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [78]:
correct_prediction = tf.equal(tf.arg_max(y_conv,1),tf.arg_max(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [79]:
sess.run(tf.initialize_all_variables())

In [ ]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy,feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
        print("step %d, train accuracy %g" %(i, train_accuracy))
    sess.run(train_step,feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

print("test accuracy %g" %sess.eval(accuracy,feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))

step 0, train accuracy 0.1
